In [1]:
import subprocess
import re
import shutil
import os

# --- 1. Setup Paths & Copy Files ---
INPUT_SUB = '/kaggle/input/santa-submission/submission.csv'
INPUT_BIN = '/kaggle/input/santa-submission/bbox3'
WORKING_DIR = '/kaggle/working/'

print("📂 Setting up environment...")

# Copy submission to working directory
if os.path.exists(INPUT_SUB):
    shutil.copy(INPUT_SUB, os.path.join(WORKING_DIR, 'submission.csv'))
    print(f"✅ Copied {INPUT_SUB} -> ./submission.csv")

# Copy executable to working directory
if os.path.exists(INPUT_BIN):
    shutil.copy(INPUT_BIN, os.path.join(WORKING_DIR, 'bbox3'))
    print(f"✅ Copied {INPUT_BIN} -> ./bbox3")

# Set executable permissions
os.chmod('./bbox3', 0o755)
print("✅ Permissions set for ./bbox3")

# --- 2. Optimizer Logic ---

def run_optimizer_and_save(command, current_best_score):
    print(f"\n🚀 Running: {command}")
    
    # Run process and capture output
    process = subprocess.Popen(
        command, 
        shell=True, 
        stdout=subprocess.PIPE, 
        stderr=subprocess.STDOUT,
        text=True
    )

    final_score = None
    
    # Read output line by line
    while True:
        line = process.stdout.readline()
        if not line and process.poll() is not None:
            break
        if line:
            print(line.strip()) # Show logs in real-time
            
            # Regex to capture "Final Score: 71.01..."
            match = re.search(r"Final Score:\s+([0-9.]+)", line)
            if match:
                final_score = float(match.group(1))

    # Logic to handle the result
    if final_score is not None:
        print(f"🏁 Run Finished. Score: {final_score}")
        
        # 1. Always save a timestamped/scored copy so nothing is lost
        backup_name = f"sub_{final_score:.6f}.csv"
        shutil.copy('submission.csv', backup_name)
        
        # 2. Update 'best_submission.csv' if it's the global best
        if final_score < current_best_score:
            print(f"🏆 NEW BEST! {current_best_score} -> {final_score}")
            shutil.copy('submission.csv', 'best_submission.csv')
            return final_score
        else:
            print(f"🔸 No improvement (Best is still {current_best_score})")
            return current_best_score
    else:
        print("⚠️ Warning: Could not detect 'Final Score' in output.")
        return current_best_score

# --- 3. Execution Loop ---

# Set your initial best score (use a high number or your current leaderboard score)
global_best = float('inf')

# Define your strategy here
commands_to_run = [
    "./bbox3 -n 555 -r 55", 
    # Add more lines if you want to loop it
]

for cmd in commands_to_run:
    global_best = run_optimizer_and_save(cmd, global_best)

print(f"\n✨ All tasks complete. Best score: {global_best}")

📂 Setting up environment...
✅ Copied /kaggle/input/santa-submission/submission.csv -> ./submission.csv
✅ Copied /kaggle/input/santa-submission/bbox3 -> ./bbox3
✅ Permissions set for ./bbox3

🚀 Running: ./bbox3 -n 555 -r 55
Loading submission.csv...
Loaded 200 configs (N=1 to N=200).
Initial Total Score: 70.979251

Starting parallel optimization on 200 configurations.
Using 4 threads (Max Concurrency).


Optimization Complete
Initial Score: 70.979251277814
Final Score:   70.979251277814
Improvement:   0.000000000000 (0.00%)
Total Time:    511.4s
Saved results to: submission.csv
🏁 Run Finished. Score: 70.979251277814
🏆 NEW BEST! inf -> 70.979251277814

✨ All tasks complete. Best score: 70.979251277814
